In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import os

In [2]:
# load data
df = pd.read_csv('sierraleone-bumbuna.csv')
df.head() # show top 5 rows

,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-10-30 00:01,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.1,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
1,2021-10-30 00:02,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
2,2021-10-30 00:03,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
3,2021-10-30 00:04,-0.7,0.0,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.1,22.3,22.6,NaN
4,2021-10-30 00:05,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN


In [4]:
# Columns to check for negative values
columns_to_clean = ['GHI', 'DNI', 'DHI', 'WS', 'WSgust']

# Replace negatives with NaN
df[columns_to_clean] = df[columns_to_clean].apply(lambda x: x.where(x >= 0, np.nan))

In [7]:
# Count missing values in key columns
missing = df[columns_to_clean + ['ModA', 'ModB']].isnull().sum()
print("Missing Values:\n", missing)

Missing Values:
 GHI       0
DNI       0
DHI       0
WS        0
WSgust    0
ModA      0
ModB      0
dtype: int64


In [6]:
# Impute with median
for col in columns_to_clean + ['ModA', 'ModB']:
    median = df[col].median()
    df[col].fillna(median, inplace=True)

In [ ]:
# Columns for outlier detection
# outlier_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# # Calculate Z-scores
# z_scores = np.abs(stats.zscore(df[outlier_columns]))

# # Flag rows with any |Z| > 3
# outlier_rows = (z_scores > 3).any(axis=1)
# print(f"Found {outlier_rows.sum()} outlier rows.")

Found 15092 outlier rows.


In [ ]:
# Replace outliers with median
# for col in outlier_columns:
#     median = df[col].median()
#     df.loc[z_scores[col] > 3, col] = median

In [28]:
# Sample data
# df = pd.read_csv(...)

outlier_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# --- STEP 1: For each column, compute its own Z-scores and replace >3 with the median ---
for col in outlier_columns:
    # compute z-scores *just* for this column
    col_z = np.abs(stats.zscore(df[col].astype(float), nan_policy='omit'))
    
    # find outlier mask
    mask = col_z > 3
    
    # count and report
    n_out = mask.sum()
    if n_out:
        print(f"Column {col!r}: replacing {n_out} outliers with median")

    # replace in df
    median = df.loc[~mask, col].median()   # median of the non-outliers is optional but common
    df.loc[mask, col] = median

# --- STEP 2: (Optional) Verify that there are no more |Z|>3 outliers left ---
# Recompute overall check if you like:
new_z = np.abs(stats.zscore(df[outlier_columns].astype(float), nan_policy='omit'))
print("Remaining outlier rows:", (new_z > 3).any(axis=1).sum())


Remaining outlier rows: 0


In [ ]:
df.to_csv('data/cleaned.csv', index=False)